<a href="https://colab.research.google.com/github/chinmay13579/Assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Identity Block**


In [1]:
import time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [2]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        #########################
        ### 1st residual block
        #########################
        """The following code uses residual blocks with skip connections such that the input passed via the shortcut matches the dimensions of the main path's output"""
        self.block_1 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=1,
                                out_channels=4,
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),
                torch.nn.BatchNorm2d(4),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=4,
                                out_channels=1,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.BatchNorm2d(1)
        )
        
        self.block_2 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=1,
                                out_channels=4,
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),
                torch.nn.BatchNorm2d(4),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=4,
                                out_channels=1,
                                kernel_size=(3, 3),
                                stride=(1, 1),
                                padding=1),
                torch.nn.BatchNorm2d(1)
        )

        #########################
        ### Fully connected
        #########################        
        self.linear_1 = torch.nn.Linear(1*28*28, num_classes)

        
    def forward(self, x):
        
        #########################
        ### 1st residual block
        #########################
        shortcut = x
        x = self.block_1(x)
        x = torch.nn.functional.relu(x + shortcut)
        
        #########################
        ### 2nd residual block
        #########################
        shortcut = x
        x = self.block_2(x)
        x = torch.nn.functional.relu(x + shortcut)
        
        #########################
        ### Fully connected
        #########################
        logits = self.linear_1(x.view(-1,  1*28*28))
        return logits
         

In [3]:
"""device loading"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""Setting parameters"""
random_seed = 123
learning_rate = 0.01
num_epochs = 10
batch_size = 128

"""Since number of output characters is 10 in MNIST dataset we have set the number of classes to be 10"""
num_classes = 10


# Note transforms.ToTensor() scales input images
# to 0-1 range
"""Below i have downloaded the MNIST fashion dataset."""
train_dataset = datasets.FashionMNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.FashionMNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())

"""Below is the DataLoader class.
   Parameters
    Dataset:
         The dataset to be loaded
    batch_size:
          Amount of samples per batch to load
    shuffle:
          it states whether to change the ordering of data at every epoch.
         """
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [4]:
    
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)
model = model.to(device)
"""Here I have used the SGD optimizer as mentioned"""   
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [5]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):            
        features = features.to(device)
        targets = targets.to(device)
        """Below we have obtained the output from our model"""
        logits = model(features)
        _, predicted_labels = torch.max(logits, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100



for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
        
        ### FORWARD AND BACK PROP
        logits = model(features)
        cost = torch.nn.functional.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 250:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    model = model.eval() # eval mode to prevent upd. batchnorm params during inference
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/469 | Cost: 2.7354
Epoch: 001/010 | Batch 250/469 | Cost: 0.5253
Epoch: 001/010 training accuracy: 78.12%
Epoch: 002/010 | Batch 000/469 | Cost: 0.7174
Epoch: 002/010 | Batch 250/469 | Cost: 0.4582
Epoch: 002/010 training accuracy: 84.74%
Epoch: 003/010 | Batch 000/469 | Cost: 0.3811
Epoch: 003/010 | Batch 250/469 | Cost: 0.4767
Epoch: 003/010 training accuracy: 84.11%
Epoch: 004/010 | Batch 000/469 | Cost: 0.4975
Epoch: 004/010 | Batch 250/469 | Cost: 0.3843
Epoch: 004/010 training accuracy: 84.15%
Epoch: 005/010 | Batch 000/469 | Cost: 0.3641
Epoch: 005/010 | Batch 250/469 | Cost: 0.4638
Epoch: 005/010 training accuracy: 86.38%
Epoch: 006/010 | Batch 000/469 | Cost: 0.3518
Epoch: 006/010 | Batch 250/469 | Cost: 0.4027
Epoch: 006/010 training accuracy: 86.63%
Epoch: 007/010 | Batch 000/469 | Cost: 0.3996
Epoch: 007/010 | Batch 250/469 | Cost: 0.4327
Epoch: 007/010 training accuracy: 84.01%
Epoch: 008/010 | Batch 000/469 | Cost: 0.4314
Epoch: 008/010 | Batch 

In [6]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 84.70%


**Convolution Block**

In [7]:
class ResidualBlock(torch.nn.Module):
    """Residual Block is used when the dimensions of the input image and the output do not match.
       Here we first perform convolutions and before applying the relu function, we make sure if the output has same dimensions as that of input.  """

    def __init__(self, channels):
        
        super(ResidualBlock, self).__init__()
        
        self.block = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=channels[0],
                                out_channels=channels[1],
                                kernel_size=(3, 3),
                                stride=(2, 2),
                                padding=1),
                torch.nn.BatchNorm2d(channels[1]),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=channels[1],
                                out_channels=channels[2],
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),   
                torch.nn.BatchNorm2d(channels[2])
        )

        self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=channels[0],
                                out_channels=channels[2],
                                kernel_size=(1, 1),
                                stride=(2, 2),
                                padding=0),
                torch.nn.BatchNorm2d(channels[2])
        )
            
    def forward(self, x):
        shortcut = x
        
        block = self.block(x)
        shortcut = self.shortcut(x)    
        x = torch.nn.functional.relu(block+shortcut)

        return x
##########################

In [8]:
##########################
### MODEL
##########################



class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        self.residual_block_1 = ResidualBlock(channels=[1, 4, 8])
        self.residual_block_2 = ResidualBlock(channels=[8, 16, 32])
    
        self.linear_1 = torch.nn.Linear(7*7*32, num_classes)

        
    def forward(self, x):

        out = self.residual_block_1(x)
        out = self.residual_block_2(out)
         
        logits = self.linear_1(out.view(-1, 7*7*32))
        return logits

    
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [9]:
""" This function is used for training our model.
    We have used optimisers which are used to update the parameters."""
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits = model(features)
        cost = torch.nn.functional.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))

    model = model.eval() # eval mode to prevent upd. batchnorm params during inference
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/468 | Cost: 2.3996
Epoch: 001/010 | Batch 050/468 | Cost: 0.7546
Epoch: 001/010 | Batch 100/468 | Cost: 0.7071
Epoch: 001/010 | Batch 150/468 | Cost: 0.6311
Epoch: 001/010 | Batch 200/468 | Cost: 0.5953
Epoch: 001/010 | Batch 250/468 | Cost: 0.4697
Epoch: 001/010 | Batch 300/468 | Cost: 0.5542
Epoch: 001/010 | Batch 350/468 | Cost: 0.4430
Epoch: 001/010 | Batch 400/468 | Cost: 0.4010
Epoch: 001/010 | Batch 450/468 | Cost: 0.4549
Epoch: 001/010 training accuracy: 84.39%
Epoch: 002/010 | Batch 000/468 | Cost: 0.5272
Epoch: 002/010 | Batch 050/468 | Cost: 0.3649
Epoch: 002/010 | Batch 100/468 | Cost: 0.4318
Epoch: 002/010 | Batch 150/468 | Cost: 0.5155
Epoch: 002/010 | Batch 200/468 | Cost: 0.3553
Epoch: 002/010 | Batch 250/468 | Cost: 0.3697
Epoch: 002/010 | Batch 300/468 | Cost: 0.5003
Epoch: 002/010 | Batch 350/468 | Cost: 0.3513
Epoch: 002/010 | Batch 400/468 | Cost: 0.5574
Epoch: 002/010 | Batch 450/468 | Cost: 0.3293
Epoch: 002/010 training accuracy: 85.73

In [10]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 87.51%
